## Hardware Testbed and Large-scale Testbed Co-simulation

## Co-Simulation

In [1]:
# --- imports ---

import os
import time
import logging
logger = logging.getLogger(__name__)

import csv
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import andes
andes.config_logger(30)
print(andes.__version__)

1.8.5


In [2]:
%matplotlib inline

In [3]:
# --- set logging level ---

logger.setLevel(logging.DEBUG)

In [4]:
# --- set path ---

path_ltb = os.getcwd()
path_case = os.path.join(path_ltb, 'case')
path_data = os.path.join(path_ltb, 'data')
datar = os.path.join(path_data, 'datar.txt')
dataw = os.path.join(path_data, 'dataw.txt')

case1 = os.path.join(path_case, 'ieee14_htb.xlsx')
case2 = os.path.join(path_case, 'pjm5_hlb.xlsx')

# --- set case ---
ss = andes.load(case1,
                no_output=True,
                default_config=False,
                setup=False)

#  --- HTB setttings ---

pq_htb = 'PQ_6'  # load represents for HTB
bus_htb = ss.PQ.get(idx=pq_htb, src='bus', attr='v')  # get bus index of HTB
bus_slack = ss.Slack.bus.v[0]  # get bus index of slack bus

# add Bus Freq. Measurement to HTB bus
ss.add('BusFreq', {'idx': 'BusFreq_HTB',
                   'name': 'BusFreq_HTB',
                   'bus': bus_htb,
                   'Tf': 0.02,
                   'Tw': 0.02,
                   'fn': 60})
ss.add('BusFreq', {'idx': 'BusFreq_Slack',
                   'name': 'BusFreq_Slack',
                   'bus': bus_slack,
                   'Tf': 0.02,
                   'Tw': 0.02,
                   'fn': 60})

ss.setup()

True

### Data I/O

Data IO from HTB to LTB is converted by a linear function:

$$ y = htb_{s} \cdot x + htb_{b} $$

where $x$ is the data from HTB (converted from ***HEX*** to ***DEC***),
$y$ is the output,$htb_{s}$ is the scale factor,
and $htb_{b}$ is the bias.

In contrast, data IO from LTB to HTB is converted by a linear function:

$$ y = htb_{s} \cdot x $$

where $x$ is the data from LTB, $y$ is the data to HTB, $ltb_{s}$ is the scale factor.

Data IO configuration:
```python
"""
Configurations
----------------
k_df: int
    default counter value
p_df: float
    default active power
q_df: float
    default reactive power
k_pu: float
    scaler of p.u., convert data from HTB base to LTB base
"""
```

In [5]:
io_config = dict(k_df=-4, p_df=0, q_df=0, htb_s=1e4, htb_b=2)

def data_read(file=datar, config=io_config):
    """
    Read data from a txt file.

    ``k``, ``p``, and ``q`` are the counter, active power,
    and reactive power, respectively.

    Parameters
    ------------
    file: str
        Name of the file to read
    config: dict
        Configuration dictionary

    Returns
    ---------
    out: list
        List of read data, [k, p, q]
    txtc: str
        Raw text read from file
    io_flag: bool
        Flag to indicate if data reading is successful
    """
    [k_df, p_df, q_df] = [io_config['k_df'], io_config['p_df'], io_config['q_df']]
    io_flag = False
    try:
        txtr = open(file)
        txtc = txtr.read()
        txtr.close()
        [k, p, q] = [int(i, 10) for i in txtc.split()]  # HEX to DEC
        # --- data conversion ---
        p = p / io_config['htb_s'] + io_config['htb_b']
        q = q / io_config['htb_s'] + io_config['htb_b']
        io_flag = True
        msg = "Data read from %s: k=%d, p=%f, q=%f" % (file, k, p, q)
    except FileNotFoundError:
        out = [k_df, p_df, q_df]
        msg = "File Not Found Error occured data read from %s error" % file
    except ValueError:
        out = [k_df, p_df, q_df]
        msg = "Value Error occured data read from %s error" % file
    logger.warning(msg)
    return out, txtc, io_flag


def data_write(dataw, file="dataw.txt", config=io_config):
    """
    Write data into a txt file.

        ``k``, ``p``, and ``q`` are the counter, active power, and reactive power, respectively.

    Parameters
    ------------
    dataw: list
        list of data to write
    file: str
        name of the file to write
    config: dict
        configuration dictionary

    Returns
    ---------
    io_flag: bool
        Flag to indicate if data writting is successful
    """
    io_flag = False
    scsv = open(file, "w")
    writer = csv.writer(scsv)
    writer.writerow([i * io_config['htb_s'] for i in dataw])
    scsv.close()
    io_flag = True
    msg = "Data write to %s: k=%d, p=%f, q=%f" % (file, dataw[0], dataw[1], dataw[2])
    logger.warning(msg)
    return io_flag


### Co-Simulation

TODO: Counter generated by HTB is a loop of integers from 11 to 199.

Counter is calculated by a linear function:

$$ k = k_{s} \cdot k_r + k_{b} $$

where $k_r$ is the counter read from HTB, $k$ is the actual counter,
$k_{s}$ is the scale factor, and $k_{b}$ is the bias.

Co-simulation status:
```python
"""
Status
-------
iter_total: int
    Number of total iterations
iter_fail: int
    Number of failed iterations
ks: int
    counter base
kb: int
    counter bias
"""
```

Co-simulation configuration:
```python
"""
Configurations
----------------
ti: float
    Starting point of TDS
t_step: float
    Time step of co-simulation
t_total: float
    Time length of co-simulation
itermax_io: int
    Maximum iteration number of data IO
load_switch: bool
    Switch of load, True for HTB load on, False for load off
"""
```

Co-simulation timeseries data:
```python
"""
Data
-----
ks: int
    counter base
kb: int
    counter bias
kr: int
    read counter
freq: float
    frequency from LTB
volt: float
    voltage from LTB
p: float
    active power from HTB
q: float
    reactive power from HTB
tw: float
    time to write
tr: float
    time to read
"""
```

In [6]:
cs_stat = dict(iter_total=0, iter_fail=0,
               ks=0, kb=0, kr=-1, k=0,
               a_ltb=0, p=0, q=0, tw=0, tr=0)

cs_config = dict(ti=1, t_step=0.05, t_total=30,
                 itermax_io=100, load_switch=True)

cs_col = ['ks', 'kb', 'kr', 'a_ltb', 'p', 'q', 'tw', 'tr']
rows = np.ceil(cs_config['t_total'] / cs_config['t_step'])
cs_num = -1 * np.ones((int(rows), len(cs_col)))


In [7]:
const_freq = 2 * np.pi * ss.config.freq  # constant to calculate bus angle
flag_init = True  # Flag to indicate the very first iteration
flag_tc0 = True  # Flag to record `tc0`

flag_datar = False  # Flag to indicate if data read is successful
flag_dataw = False  # Flag to indicate if data write is successful

In [ ]:
# --- system initial conditions ---

a0 = ss.Bus.get(idx=bus_slack, src='a', attr='v')  # initial slack bus angle
p0 = ss.PQ.get(idx=pq_htb, src='p0', attr='v')  # initial HTB bus active power
q0 = ss.PQ.get(idx=pq_htb, src='q0', attr='v')  # initial HTB bus reactive power

ss.TDS.config.no_tqdm = 1  # turn off tqdm progress bar
ss.TDS.config.criteria = 0  # turn off stability criteria

# set constant power load
ss.PQ.config.p2p = 1
ss.PQ.config.q2q = 1
ss.PQ.config.p2z = 0
ss.PQ.config.q2z = 0
ss.PQ.pq2z = 0

ss.PFlow.run()  # solve power flow
ss.TDS.config.tf = cs_config['ti']
ss.TDS.run()

t0_htb = cs_stat['k'] * cs_config['t_step']
tf_htb = t0_htb + cs_config['t_step']

while cs_stat['k'] <= rows:
    if init_flag:
        while (cs_stat['kr'] != 11):
            # --- repeat reading data until kr==11 ---
            [cs_stat['kr'], cs_stat['p'], cs_stat['q']], txtc, flag_datar = data_read(file=datar, config=io_config)
            kr0 = cs_stat['kr']
            # --- reset io_config default values ---
            [io_config['p_df'], io_config['q_df']] = [cs_stat['p'], cs_stat['q']]
        init_flag = False  # Turn off init_flag after first iteration
        logger.critical("LTB ready, strat HTB to continue.")
    if cs_stat['k'] > 10:
        if init_flag:
            tc0 = time.time()  # record clock time
            time_flag = False
        # --- data read ---
        # NOTE: repeat reading data until counter update
        iter_read = 0
        while (cs_stat['kr'] != kr0 + 1) & (iter_read <= cs_config['itermax_io']):
            [cs_stat['kr'], cs_stat['p'], cs_stat['q']], txtc, flag_datar = data_read(file=datar, config=io_config)
            [io_config['p_df'], io_config['q_df']] = [cs_stat['p'], cs_stat['q']]
            iter_read += 1
            # update counter base
            if (cs_stat['kr'] == 199):
                cs_stat['ks'] += 1
                flag_base = False
                msg = "Counter base updated: %d" % cs_stat['ks']
                logger.warning(msg)
        kr0 = cs_stat['kr']  # update kr0
        tc1 = time.time()  # record clock time
        # --- LTB sim ---
        if (cs_stat['kr'] != kr0 + 1):
            if np.mod(cs_stat['k'], 20) == 0):
                msg = "Counter update: k=%d" % cs_stat['k']
                logger.warning(msg)
            # --- send data to HTB ---
            # Make sure `BusFreq` is connected to the load bus
            f_send = ss.BusFreq.get(idx='BusFreq_HTB', src='f', attr='v')  # p.u.
            v_bus = ss.Bus.get(idx=bus_htb, src='v', attr='v')  # RMS, p.u.
            dataw = [v_bus, f_send]  # LTB: voltage, angle
            data_write(dataw=dataw, file="dataw.txt", config=io_config)
            tc2 = time.time()  # send end time
            # --- LTB simulation ---
            p_inj = cs_stat['load_switch'] * cs_stat['p']
            q_inj = cs_stat['load_switch'] * cs_stat['q']
            # a) set PQ data in LTB
            ss.PQ.set(value=p_inj + p0, idx=pq_htb, src='Ppf', attr='v')
            ss.PQ.set(value=q_inj + q0, idx=pq_htb, src='Qpf', attr='v')
            # b) TDS
            ss.TDS.config.tf += cs_config['t_step']
            ss.TDS.run()
            tc3 = time.time()  # record clock time
            cs_stat['k'] = cs_stat['ks'] * cs_stat['kr'] + cs_stat['kb']
            # NOTE: tf_htb is the end time of last round
            if tc3 -  tf_htb > cs_config['t_step']:
                cs_stat['iter_fail'] += 1
            cs_stat['tr'] = tc1 - tf_htb  # read time
            cs_stat['tw'] = tc2 - tc1  # write time
            # update HTB time
            t0_htb = cs_stat['k'] * cs_config['t_step']
            tf_htb = t0_htb + cs_config['t_step']
            cs_stat['iter_total'] += 1
            #  --- record data ---
            for col in cs_col:
                cs_num[cs_stat['k'], cs_col.index(col)] = cs_stat[col]


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 8), dpi=100)
ax[0].scatter(x=range(len(tl)), y=tl)
ax[0].set_ylim([-0.01, 0.06])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 8), dpi=100)
ax[0].scatter(x=range(len(tcl)), y=trl)
ax[0].scatter(x=range(len(tcl)), y=twl)
ax[0].scatter(x=range(len(tcl)), y=tsl)
ax[0].scatter(x=range(len(tcl)), y=tcl)
ax[0].legend(['Read', 'Write', 'Sim', 'Total'])
ax[0].axhline(t_step, color='tab:red')
ax[0].set_xlim([0, len(tcl)])
ax[0].set_title("Data read time interval")
ax[0].set_ylabel("Time [s]")
ax[0].set_ylim([-0.05, 0.12])

ax[1].scatter(x=range(len(crl)), y=crl)
ax[1].set_xlim([0, len(crl)])
ax[1].set_title("Read counter")
ax[1].set_ylabel("Number")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5), dpi=400)

ss.TDS.plt.plot(ss.Bus.v, a=(8),
                ax=ax[0], fig=fig,
                legend=False, show=False,
                title='Load bus voltage',
                ylabel='Voltage [p.u.]')

ss.TDS.plt.plot(ss.GENCLS.omega,
                ax=ax[1], fig=fig,
                legend=False, show=False,
                ytimes=ss.config.freq,
                title='Generator omega',
                ylabel='Frequency [Hz]')

In [ ]:
crl

In [ ]:
plt.plot(range(len(crl)), crl)
plt.title("Counter")
plt.xlabel("Seqence")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5), dpi=400)

# ax[2].scatter(x=np.array(tl), y=np.array(fsl))
ax[0].plot(np.array(tl) - tl[0], np.array(fsl))
# ax[2].set_xlim([0, len(asl)])
ax[0].set_xlim([0, len(crl) * t_step])
ax[0].set_title("Freq. send")
ax[0].set_ylabel("p.u.")
ax[0].set_xlabel("Time [s]")

ax[1].plot(np.array(tl) - tl[0], np.array(vsl))
# ax[2].set_xlim([0, len(asl)])
ax[1].set_xlim([0, len(crl) * t_step])
ax[1].set_title("Volt. send")
ax[1].set_ylabel("p.u.")
ax[1].set_xlabel("Time [s]")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5), dpi=400)

# ax[2].scatter(x=np.array(tl), y=np.array(fsl))
ax[0].plot(np.array(tl) - tl[0], np.array(prl))
# ax[2].set_xlim([0, len(asl)])
ax[0].set_xlim([0, len(crl) * t_step])
ax[0].set_title("P read")
ax[0].set_ylabel("p.u.")
ax[0].set_xlabel("Time [s]")


ax[1].plot(np.array(tl) - tl[0], np.array(qrl))
# ax[2].set_xlim([0, len(asl)])
ax[1].set_xlim([0, len(crl) * t_step])
ax[1].set_title("Q read")
ax[1].set_ylabel("p.u.")
ax[1].set_xlabel("Time [s]")

In [ ]:
ss.dae.ts.y[:, ss.Bus.v.a[8]]

In [ ]:
data1 = pd.DataFrame()
data1['fs'] = fsl
data1['vs'] = vsl
data1['pr'] = prl
data1['qr'] = qrl

data2 = pd.DataFrame()
data2['busv'] = ss.dae.ts.y[:, ss.Bus.v.a[8]]
data2['wg1'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[0]]
data2['wg2'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[1]]
data2['wg3'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[2]]
data2['wg4'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[3]]
data2['wg5'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[4]]

data1.to_csv('data1_2.csv', index=False)
data2.to_csv('data2_2.csv', index=False)